# 1

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

## 1.1

In [ ]:

df = pd.read_csv('/content/Electric_Production.csv', parse_dates=['DATE'], index_col='DATE')
result= seasonal_decompose(df['IPG2211A2N'], model='additive', period=12)


## 1.2

In [ ]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    subplot_titles=('Observed', 'Trend', 'Seasonal', 'Residual'),
                    vertical_spacing=0.05)
fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, mode='lines', name='Observed', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines', name='Trend', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines', name='Seasonal', line=dict(color='orange')), row=3, col=1)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines', name='Residual', line=dict(color='red')), row=4, col=1)
fig.update_layout(height=800, width=900, title_text="Time Series Decomposition", showlegend=False)
fig.show()

## 1.3

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, mode='lines', name='Observed', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines', name='Trend', line=dict(color='green', dash='dash')))
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines', name='Seasonal', line=dict(color='orange', dash='dot')))
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines', name='Residual', line=dict(color='red', dash='dashdot')))
fig.update_layout(title='Combined Time Series Components',
                  xaxis_title='Date',
                  yaxis_title='Electric Production',
                  legend_title='Components',
                  height=600, width=900)

fig.show()

## 1.4

In [ ]:

model = ARIMA(df['IPG2211A2N'], order=(5, 1, 0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=12)
forecast_index = pd.date_range(df.index[-1], periods=12, freq='M')
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['IPG2211A2N'], mode='lines', name='Observed', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=forecast_index, y=forecast, mode='lines', name='Forecast', line=dict(color='red', dash='dash')))
fig.update_layout(title='Electric Production Forecast',
                  xaxis_title='Date',
                  yaxis_title='Electric Production',
                  legend_title='Data',
                  height=600, width=900)

fig.show()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

<ipython-input-13-37e241c1db40>:7: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



# 2